In [1]:
import sys
sys.path.append('/home/fac_files/code/collective_effects/')
%load_ext autoreload
%autoreload 1

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import timeit
import cProfile
mu0 = 4*np.pi*1e-7
c   = 299792458
ep0 = 1/c**2/mu0

In [4]:
%aimport collective_effects
%aimport impedances

In [5]:
ring = collective_effects.Ring()

In [8]:
w       = np.logspace(4,12.4,3000)
w       = np.array([-np.flipud(w),w]).ravel()

epb     = np.array([1, 1, 1, 1])
mub     = np.array([1, 1, 1, 1])
ange    = np.array([0, 0, 0, 0])
angm    = np.array([0, 0, 0, 0])
sigmadc = np.array([0, 4e6, 5.9e7, 1])
tau     = np.array([0, 0, 1, 0],dtype=float) * 27e-15
epr,mur = impedances._prepare_input_epr_mur(w,epb,mub,ange,angm,sigmadc,tau)

b       = np.array([12.000, 12.001, 13.000])*1e-3
L       = 480
   
Zl, Zv, Zh = impedances.resistive_multilayer_round_pipe(w,epr,mur,b,L,ring.E)

In [9]:
Zl = impedances.longitudinal_resonator(np.array([7e3]),np.array([1]),np.array([10e9]),w)
Zv = impedances.transverse_resonator(np.array([10e6]),np.array([1]),np.array([20e9]),w)

In [11]:
plt.loglog(w,Zv.real,'b')
plt.loglog(w,Zv.imag,'g')
plt.loglog(w,-Zv.imag,'g--')
plt.show()

/usr/local/lib/python3.4/dist-packages/matplotlib/backends/backend_gtk3.py:215: Warning: Source ID 295 was not found when attempting to remove it
  GLib.source_remove(self._idle_event_id)


In [12]:
#%timeit -r 1 delta = ring.longitudinal_mode_coupling(w,Zl,n_rad=12,n_azi=12,mu = 0)
ring.nbun = 864
#cProfile.run("delta = ring.longitudinal_mode_coupling(w,Zl,n_rad=4,n_azi=3,mu = 0)")
#cProfile.run("delta = ring.transverse_mode_coupling(w,Zv,n_rad=4,n_azi=3,mu = 0)")
delta = ring.transverse_mode_coupling(w,Zv,n_rad=4,n_azi=3,mu = 0)

In [13]:
gr = np.array(delta.imag)
gr.sort(axis=1)
tush = np.array(delta.real)
tush.sort(axis=1)

In [16]:
plt.plot(ring.cur_bun,tush)
plt.show()

/usr/local/lib/python3.4/dist-packages/matplotlib/backends/backend_gtk3.py:215: Warning: Source ID 1334 was not found when attempting to remove it
  GLib.source_remove(self._idle_event_id)


In [166]:
n_rad, n_azi = 1, 1

a = np.zeros([2*n_azi+1,n_rad+1,2*n_azi+1,n_rad+1])
b = np.zeros([(2*n_azi+1)*(n_rad+1),(2*n_azi+1)*(n_rad+1)])
i=0
for m in range(n_azi+1):
    for k in range(n_rad+1):
        for n in range(n_azi+1):
            for l in range(n_rad+1):
                a[n_azi+m,k,n_azi+n,l] = i
                b[n_azi+m +k*(2*n_azi+1),n_azi+n +l*(2*n_azi+1)] = i
                i+=1
                a[n_azi+m,k,n_azi-n,l] = i
                b[n_azi+m +k*(2*n_azi+1),n_azi-n +l*(2*n_azi+1)] = i
                i+=1
                a[n_azi-m,k,n_azi-n,l] = i
                b[n_azi-m +k*(2*n_azi+1),n_azi-n +l*(2*n_azi+1)] = i
                i+=1
a.swapaxes(0,3).swapaxes(1,2).reshape([(n_rad+1)*(2*n_azi+1),(n_rad+1)*(2*n_azi+1)]).transpose()

array([[ 32.,  26.,   0.,  35.,  29.,   0.],
       [  8.,   2.,   6.,  11.,   5.,   9.],
       [ 31.,  25.,  30.,  34.,  28.,  33.],
       [ 44.,  38.,   0.,  47.,  41.,   0.],
       [ 20.,  14.,  18.,  23.,  17.,  21.],
       [ 43.,  37.,  42.,  46.,  40.,  45.]])

In [167]:
b

array([[ 32.,  26.,   0.,  35.,  29.,   0.],
       [  8.,   2.,   6.,  11.,   5.,   9.],
       [ 31.,  25.,  30.,  34.,  28.,  33.],
       [ 44.,  38.,   0.,  47.,  41.,   0.],
       [ 20.,  14.,  18.,  23.,  17.,  21.],
       [ 43.,  37.,  42.,  46.,  40.,  45.]])

In [184]:
swap_reshape = lambda A: A.swapaxes(0,3).swapaxes(1,2).reshape([(n_rad+1)*(2*n_azi+1),(n_rad+1)*(2*n_azi+1)]).transpose()
#%timeit c = swap_reshape(a)
#%timeit c = a.swapaxes(0,3).swapaxes(1,2).reshape([(n_rad+1)*(2*n_azi+1),(n_rad+1)*(2*n_azi+1)]).transpose()

True